In [2]:
import numpy as np

In [3]:
# problem 2

def verify_r1cs(A : np.ndarray, B : np.ndarray, C : np.ndarray, w : np.ndarray):
    '''
    A, B, C are matrices with shape (n_equations, n_variables)
    w is a vector with shape (n_variables,)
    returns True if Aw * Bw = Cw, False otherwise
    '''
    left = np.dot(A, w) * np.dot(B, w)
    
    print("left", left)
    right = np.dot(C, w)
    print("right", right)
    return np.allclose(left, right)
    
    
A = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0]
])

B = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1]
])

C = np.array([
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [36, 0, 0, 0, -36, 0, 11]
])


def parameterize_witness(x, y):
    a = x*x
    b = x*y
    c = y*y
    d = a*c
    
    return np.array([1, x, y, a, b, c, d])

# The witness vector W
W = parameterize_witness(3, 3)


print("A.shape: ", A.shape)
print("B.shape: ", B.shape)
print("C.shape: ", C.shape)
print("W.shape: ", W.shape)

print(verify_r1cs(A, B, C, W))


A.shape:  (5, 7)
B.shape:  (5, 7)
C.shape:  (5, 7)
W.shape:  (7,)
left [  9   9   9  81 729]
right [  9   9   9  81 603]
False


In [5]:
# problem 3
from py_ecc.bn128 import pairing, multiply, G1, G2, curve_order, field_modulus, add, eq
from py_ecc.fields import bn128_FQ2, bn128_FQ
from typing import Any
import numpy as np

In [11]:
G2 + G2

((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634),
 (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531),
 (10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634),
 (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531))

In [10]:


def is_2d_point(point: Any) -> bool:
    if point is None:
        return True  # Point at infinity is considered a valid 2D point
    if isinstance(point, tuple):
        if isinstance(point[0], bn128_FQ2):
            return True
    else:
        raise ValueError(f"Invalid point {point}")
    return False

def matrix_point_dot_prod(M : np.ndarray, s : np.ndarray) -> list[tuple]:
    '''
    M is a matrix with shape (n_equations, n_variables)
    s is a vector with shape (n_variables,)
    returns a vector with shape (n_equations,)
    '''
    assert M.shape[1] == s.shape[0]
    
    out = []
    for i in range(M.shape[0]):
        buf = None # this is the identity element of the group
        for j in range(M.shape[1]):
            elm = multiply(s[j], M[i, j])
            buf = add(buf, elm)
            
        out.append(tuple(buf))
    return out
            
def hadamard_prod(a : list, b : list) -> list:
    '''
    a, b are vectors with shape (n_variables,)
    returns a vector with shape (n_variables,)
    '''
    assert len(a) == len(b)
    prod = []
    for i in range(len(a)):
        # we need to ensure the G2 point is first
        if is_2d_point(tuple(a[i])):
            print("a 2d")
            prod.append(pairing(a[i], b[i]))
        elif is_2d_point(tuple(b[i])):
            print("b 2d")
            prod.append(pairing(b[i], a[i]))
        else:
            raise ValueError(f"Invalid point got two 1d points {a[i]} and {b[i]}")
    return prod
    
L = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

R = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

S = np.array([
    multiply(G1, 1),
    multiply(G2, 1),
    multiply(G1, 1),
])

S2 = np.array([
    multiply(G2, 1),
    multiply(G1, 1),
    multiply(G2, 1),
])

a = matrix_point_dot_prod(L, S2)
b = matrix_point_dot_prod(R, S)

c = hadamard_prod(a, b)
print(c)

a 2d
b 2d
a 2d
[(18443897754565973717256850119554731228214108935025491924036055734000366132575, 10734401203193558706037776473742910696504851986739882094082017010340198538454, 5985796159921227033560968606339653189163760772067273492369082490994528765680, 4093294155816392700623820137842432921872230622290337094591654151434545306688, 642121370160833232766181493494955044074321385528883791668868426879070103434, 4527449849947601357037044178952942489926487071653896435602814872334098625391, 3758435817766288188804561253838670030762970764366672594784247447067868088068, 18059168546148152671857026372711724379319778306792011146784665080987064164612, 14656606573936501743457633041048024656612227301473084805627390748872617280984, 17918828665069491344039743589118342552553375221610735811112289083834142789347, 19455424343576886430889849773367397946457449073528455097210946839000147698372, 7484542354754424633621663080190936924481536615300815203692506276894207018007), (1844389775456597371725685011955473122821

In [ ]:

def verify_r1cs_with_points(
    L : np.ndarray, 
    R : np.ndarray, 
    O : np.ndarray, 
    S_1 : np.ndarray,
    S_2 : np.ndarray,
    G : np.ndarray
):
    '''
    L, R, O are matrices with shape (n_equations, n_variables)
    s is a vector with shape (n_variables,)
    returns True if L*s \odot R*s = O*s, False otherwise
    '''
    #check L, R, O are all the same shape and field elements
    assert L.shape == R.shape == O.shape 
    assert np.max(L) <= field_modulus
    assert np.max(R) <= field_modulus
    assert np.max(O) <= field_modulus
    
    #check s is a valid witness
    assert S_1.shape[0] == S_2.shape[0] == G.shape[0] == L.shape[1]
    
    left_a = matrix_point_dot_prod(L, S_1)
    left_b = matrix_point_dot_prod(R, S_2)
    left = hadamard_prod(left_a, left_b)
    right_a = matrix_point_dot_prod(O, S_1)
    right = hadamard_prod(right_a, G)
    
    for a, b in zip(left, right):
        if not eq(a, b):
            return False
    return True

O = np.array([
    [0,1, 0],
    [0,1, 0],
    [1, 0, 0]
])

L = np.array([
    [0, 1, 0],
    [0, 0, 1],
    [0, 1, 0]
])

R = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

S_1 = np.array([
    multiply(G1, 1),
    multiply(G1, 1),
    multiply(G1, 1),
])

S_2 = np.array([
    multiply(G2, 1),
    multiply(G2, 1),
    multiply(G2, 1),
])

G = np.array([
    multiply(G2, 1),
    multiply(G2, 1),
    multiply(G2, 1),
])

print(verify_r1cs_with_points(L, R, O, S_1, S_2, G))


In [ ]:
G = np.array([
    multiply(G2, 1),
    multiply(G2, 1),
    multiply(G2, 1),
])


S_1 = np.array([
    multiply(G1, 1),
    multiply(G1, 1),
    multiply(G1, 1),
])





right_a = matrix_point_dot_prod(O, S_1)
right = hadamard_prod(right_a, G)
print(right)

In [ ]:

def check_s_equality(sG1, sG2):
    '''
    sG1, sG2 are points on the curve
    returns True if sG1 == sG2, False otherwise
    '''
    G1_ = multiply(G1, 1)
    G2_ = multiply(G2, 1)   
    return eq(pairing(G2_, sG1), pairing(sG2, G1_))



Hint: When you get s encrypted with both G1 and G2 generators, you
don’t know whether or not they have the same discrete logarithm. However, it
is straightforward to check using another equation. Figure out how to discover
if sG1 == sG2 if you are given the elliptic curve points but not s.
Solidity cannot multiply G2 points, do this assignment in Python